In [1]:
from google.colab import auth
import gspread
from google.auth import default
import pandas
import random
from numpy import int64

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
owner_emails = ['nickreid@gmail.com', 'pushkala.jayaraman@icahn.mssm.edu', 'gpratishtha17@gmail.com', 'teya.dragovic@gmail.com']

def get_or_create_spreadsheet(worksheet_name):
  try:
    worksheet = gc.open(worksheet_name)
  except gspread.exceptions.SpreadsheetNotFound:
    worksheet = gc.create(worksheet_name)
    for _email in owner_emails:
      worksheet.share(_email, perm_type='user', role='writer')
  return worksheet

def write_A1_query(df, offset=1):
  cols = [
    'ABCDEFGHIJKLMNOPQURSTUVWXYZ'[index]
    for index, col in enumerate(df.columns)
  ]
  return 'A{}:{}{}'.format(
    offset,
    cols[-1],
    len(df)+offset
  )

def write_worksheet(df, spreadsheet, worksheet_name):
  df = remove_int64(df)
  worksheet = spreadsheet.worksheet(worksheet_name)
  worksheet.update(
      values = [
        df.columns.values.tolist()
      ] + [
        [
          value if not pandas.isnull(value) else ''
          for value in row
        ]
        for row in df.values.tolist()
      ],
      range_name = write_A1_query(df)
  )

def remove_int64(df):
  return pandas.DataFrame(
    [
        [
            value if type(value) is not int64 else int(value)
            for value in row
        ]
        for row in df.values
    ],
    index = df.index,
    columns = df.columns
  )

In [ ]:
## Update overview page
yir_spreadsheet = gc.open('2026 TBI YIR Spreadsheet')
overview = pandas.DataFrame(
    yir_spreadsheet.worksheet('Overview').get_all_records()
)
articles = pandas.DataFrame(
    yir_spreadsheet.worksheet('Articles').get_all_records()
)
screened_articles = pandas.DataFrame(
    yir_spreadsheet.worksheet('Screened Articles').get_all_records()
)
screening_assignments = pandas.DataFrame(
    yir_spreadsheet.worksheet('Screening Assignments').get_all_records()
)
reviewed_articles = pandas.DataFrame(
    yir_spreadsheet.worksheet('Reviewed Articles').get_all_records()
)

for priority in articles['Priority'].unique():
  priority_pmids = articles[articles['Priority']==priority]['PMID']

  search_results_index = overview[''].tolist().index('Search results')
  overview.loc[search_results_index, priority] = len(priority_pmids)

  screened_index = overview[''].tolist().index('Screened')
  overview.loc[screened_index, priority] = sum(
    (screened_articles['PMID'].isin(priority_pmids))
    & (screened_articles['Included']!='')
  )

  included_index = overview[''].tolist().index('Included')
  overview.loc[included_index, priority] = sum(
    (screened_articles['PMID'].isin(priority_pmids))
    & (screened_articles['Included']=='Yes')
  )

  excluded_index = overview[''].tolist().index('Excluded')
  overview.loc[excluded_index, priority] = sum(
    (screened_articles['PMID'].isin(priority_pmids))
    & (screened_articles['Included']=='No')
  )

  conflicts_index = overview[''].tolist().index('Conflicts')
  overview.loc[conflicts_index, priority] = sum(
    (screened_articles['PMID'].isin(priority_pmids))
    & (screened_articles['Included']=='Conflict')
  )

  assigned_for_review_index = overview[''].tolist().index('Assigned for review')
  overview.loc[assigned_for_review_index, priority] = sum(
      (reviewed_articles['PMID'].isin(priority_pmids))
      & (reviewed_articles['Reviewed By'] != "")
  )

  articles_reviewed_index = overview[''].tolist().index('Articles reviewed')
  overview.loc[articles_reviewed_index, priority] = sum(
      (reviewed_articles['PMID'].isin(priority_pmids))
      & (reviewed_articles['Informatics Novelty'] != "")
      & (reviewed_articles['Application Importance'] != "")
      & (reviewed_articles['Presentability'] != "")
  )

write_worksheet(overview, yir_spreadsheet, 'Overview')